## Homework 3 - Which book would you recomend?

-------------

### Importing related libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import os
import re

## 1. Data collection

#### 1.1. Get the list of books

* From [best books ever list](https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1) we want to collect the url associated to each book in the list and retrieve only the urls of the books listed in the first 300 pages.



* The output of this step is a `.txt` file whose single line corresponds to a book's url.

In [ ]:
#inizialize an empty list
url = []

#for each page, save the corresponding web page, find the anchor elements and save the corresponding tags  
for i in range(1, 301):
    page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+ str(i))
    soup = BeautifulSoup(page.content, features='lxml')
    tag_a = soup.find_all('a', {"class": "bookTitle"}, itemprop = "url")
    
#for each book, save the corresponding url into the array
    for j in range(0,100):
        url.append("https://www.goodreads.com"+ tag_a[j]['href'])
        
#create a txt file where for each row there is a book's url 
with open("url.txt", 'w') as f:
    f.write("\n".join(map(str, url)))
    

#### 1.2. Crawl books

1. Download the html corresponding to each of the collected urls.


2. After you collect a single page, immediatly save its `html` in a file. In this way, if your program stops, for any reason, you will not loose the data collected up to the stopping point.


3. Organize the entire set of downloaded `html` pages into folders. Each folder will contain the `htmls` of the books in page 1, page 2, ... of the list of books.

# Warning :

### Do not run the below cell because it'll download over 20 GB files !
#### Before running modify the `for` loop range to a lower number (e.g. 3) 

In [ ]:
#First we open the url.txt file, reading the lines and then close the file
f = open("url.txt", "r")
lines = f.readlines()
f.close()

#Setting our parent directory and the directory associated with each page number from the list
## PLEASE CHANGE THE PARENT DIRECTORY ACCORDING TO YOUR SYSTEM
parent_dir = "C:/Users/engme/OneDrive/Desktop/ALL Materials/Data Science - Sapienza/1st Semester/ALGORITHMIC METHODS OF DATA MINING AND LABORATORY/Labs/6/Links"
directory = "Page_"
page_num = 0


#Looping for 300 times that corresponds to the number of pages
for i in range(300):
    #Incrementing page number according to each page
    page_num += 1
    #Setting the current working directory
    directory = "Page_" + str(page_num)
    #Setting the main path to create the directory
    path = os.path.join(parent_dir, directory)
    #Creating new directory
    os.makedirs(path)
    
    #Looping for 100 times, which is the number of articles per page
    for i in range(100):
        #Selecting the corresponding link
        link = lines[i][:-1]
        #Dowloading the article
        r = requests.get(link, allow_redirects=True)
        #Setting the name file to keep the track
        file_name = "article_" + str(i+1) + ".html"
        #Saving the html file to its corresponding directory
        open(parent_dir + "/" + directory + "/" + file_name, 'wb').write(r.content)

-----------

#### 1.3 Parse downloaded pages

* Extracting the books information for each book as following:

1. Title (to save as `bookTitle`)
2. Series (to save as `bookSeries`)
3. Author(s), the first box in the picture below (to save as `bookAuthors`)
4. Ratings, average stars (to save as `ratingValue`)
5. Number of givent ratings (to save as `ratingCount`)
6. Number of reviews (to save as `reviewCount`)
7. The entire plot (to save as `Plot`)
8. Number of pages (to save as `NumberofPages`)
9. Published (Publishing Date)
10. Characters
11. Setting
12. Url

### Note to colleages: (Temporary)

* The `Url` will be added after
* The `article_i.tsv` file will be created after

In [ ]:
#Book Title
bookTitle = soup.find_all('h1', id = "bookTitle")[0].contents[0].replace('\n', '').strip()

#Book Series
series_index = False
for stri in lst:
    if 'Series' in stri:
        series_index = lst.index(stri)

bookSeries = ""

if character_index:

            bookSeries += str(soup.find_all('div', {"class": "infoBoxRowItem"})[series_index].contents[1]).strip().split(">")[1][:-3]

        

#Book Authors
bookAuthors = soup.find_all('a', {"class": "authorName"})[0].contents[0].contents[0]

#Rating Value
ratingValue = soup.find_all('span', itemprop = "ratingValue")[0].contents[0].replace('\n', '').strip()

#Rating Counts
ratingCount = soup.find_all('a', href="#other_reviews")[0].contents[2].replace('\n', '').strip().split()[0]

#Review Counts
reviewCount = soup.find_all('a', href="#other_reviews")[1].contents[2].replace('\n', '').strip().split()[0]



#Plot

#first of all, define the full description 
#(which may contain italics, bolds and preliminar descriptions not related to the plot)
full_descr = soup.find_all('div', id="description")[0].contents[3].contents

#if the description begins with this kind of description, remove it
if "<i>" in str(full_descr[0]):
    
    full_descr = full_descr[3:]  

#for every line of the description, add to the list Plot all of the lines except the ones corresponding to the <br/> tags 
Plot = []

for i in range(len(full_descr)):
    if str(full_descr[i]) != '<br/>':
        Plot.append(full_descr[i])


#for every line of the plot, remove the <i> and </i> tags
for i in range(len(Plot)):
    if '<i>' in str(Plot[i]):
        Plot[i] = re.sub(r'<i>', '', str(Plot[i]))
        Plot[i] = re.sub(r'</i>', '', str(Plot[i]))

#if the plot begins in bold, remove the <b> tags
if "<b>" in str(Plot[0]):
    Plot[0] = Plot[0].contents[0]
    
Plot = " ".join(Plot)



#Number of Pages
NumberofPages = soup.find_all('span', itemprop="numberOfPages")[0].contents[0]

#Published
Published = soup.find_all('div', {"class": "row"})[1].contents[0].replace('\n', '').strip().split(" "*8)[1]





#Characters
lst = soup.find_all('div', {"class": "infoBoxRowTitle"})

#Finding the position of Characters in "InfoBox" whether it exists or not
character_index = False
for stri in lst:
    if 'Characters' in stri:
        character_index = lst.index(stri)

#Initiating the characters list
characters = []

#If character exists, we will find them up to 5 characters.
if character_index:

    if int(len(soup.find_all('div', {"class": "infoBoxRowItem"})[character_index].contents)/2) <= 5 :
        for i in range(int(len(soup.find_all('div', {"class": "infoBoxRowItem"})[character_index].contents)/2)):
            
            #Using 2i+1 as each character is on odd index contents
            characters.append(str(soup.find_all('div', {"class": "infoBoxRowItem"})[character_index].contents[2*i+1]).strip().split(">")[1][:-3])
    else:
        for i in range(5):
            characters.append(str(soup.find_all('div', {"class": "infoBoxRowItem"})[character_index].contents[2*i+1]).strip().split(">")[1][:-3])

    characters = ", ".join(characters)

    
    

#Setting
setting = []

#Finding the index of Setting whether it exists or not !
setting_index = False
for stril in lst:
    if 'Setting' in stril:
        setting_index = lst.index(stril)

#Same approach as for characters
if setting_index:

    for i in range(3):

        setting.append(str(soup.find_all('div', {"class": "infoBoxRowItem"})[setting_index].contents[2*i+1]).strip().split(">")[1][:-3])

    setting = " ".join(setting)

